# Prepare data

In [21]:
import pandas as pd
from pathlib import Path
import numpy as np
import random
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


Перейти по ссылке

Открой в браузере:

https://www.kaggle.com/bhavikardeshna/amazon-customerreviews-polarity

Скачать датасет

Нажми кнопку "Download" (обычно в правом верхнем углу)

Получишь файл: например amazon-customerreviews-polarity.zip

Разрахивировать `test.csv` в папку проекта

In [24]:
INPUT_FILE = "test.csv"
OUTPUT_FILE = Path(__file__).parent / "reviews.csv"

df = pd.read_csv(
    INPUT_FILE,
    header=None,
    names=["label", "title", "content"]
)

required_cols = {"title", "content", "label"}
if not required_cols.issubset(df.columns):
    raise ValueError(f"CSV must contain columns: {required_cols}")

df["text"] = df["title"].astype(str) + ". " + df["content"].astype(str)

def gen_rating(label):
    return random.choice([4, 5]) if label == 1 else random.choice([1, 2])

df["rating"] = df["label"].apply(gen_rating)

start = pd.Timestamp("2024-01-01")
end = pd.Timestamp("2024-07-01")

df["timestamp"] = pd.to_datetime(
    np.random.randint(
        start.value // 10**9,
        end.value // 10**9,
        size=len(df)
    ),
    unit="s"
)

categories = ["music", "movies", "books", "electronics"]
df["product_category"] = np.random.choice(categories, size=len(df))
df["user_id"] = np.random.randint(10_000, 500_000, size=len(df))

df["review_id"] = range(1, len(df) + 1)

df = df[
    [
        "review_id",
        "user_id",
        "product_category",
        "text",
        "rating",
        "timestamp"
    ]
]



df.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Saved {len(df)} rows to {OUTPUT_FILE}")


KaggleApiHTTPError: 404 Client Error.

Resource not found at URL: https://www.kaggle.com/datasets/bhavikardeshna/amazon-customerreviews-polarity/versions/1
The server reported the following issues: Data not found
Please make sure you specified the correct resource identifiers.

# Train model

In [18]:
df = pd.read_csv("reviews.csv")

X = df["text"]
y = (df["rating"] >= 4).astype(int)

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words="english"
)

X_vec = vectorizer.fit_transform(X)

model = LogisticRegression(max_iter=1000)
model.fit(X_vec, y)

joblib.dump(model, "backend/consumer/sentiment_model.joblib")
joblib.dump(vectorizer, "backend/consumer/vectorizer.joblib")

✅ Model saved
